In [1]:
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa
import numpy as np
from typing import Dict, List
import time


/Users/krittinnagar/Speech Dubbing /.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/krittinnagar/Speech Dubbing /.venv/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Users/krittinnagar/Speech Dubbing /.venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
# import sounddevice as sd
# import soundfile as sf
# import os

# # First install: pip install sounddevice

# # Create folder
# os.makedirs("test_audio", exist_ok=True)

# # Hindi sentences to record
# sentences = [
#     "नमस्ते, मेरा नाम राज है",
#     "आज मौसम बहुत अच्छा है",
#     "मुझे हिंदी बोलना पसंद है",
#     "यह एक परीक्षण वाक्य है",
#     "भारत एक सुंदर देश है"
# ]

# print("Record 5 Hindi sentences (5 seconds each)\n")

# if os.environ.get("SKIP_AUDIO_RECORDING", "0") == "1":
#     print("⏭️ Skipping interactive microphone recording in automated mode.")
# else:
#     for i, sentence in enumerate(sentences):
#         print(f"\nSample {i+1}: {sentence}")
#         input("Press Enter when ready to record...")
#         print("🔴 Recording... (5 seconds)")
        
#         # Record
#         audio = sd.rec(int(5 * 16000), samplerate=16000, channels=1)
#         sd.wait()
        
#         # Save audio
#         sf.write(f"test_audio/sample_{i+1}.wav", audio, 16000)
        
#         # Save text
#         with open(f"test_audio/sample_{i+1}.txt", "w", encoding="utf-8") as f:
#             f.write(sentence)
        
#         print(f"✓ Saved sample_{i+1}.wav")

#     print("\n✅ Done! You now have 5 Hindi audio samples in 'test_audio/' folder")

In [3]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
from pathlib import Path

print("Loading model...")
processor = Wav2Vec2Processor.from_pretrained("ai4bharat/indicwav2vec-hindi")
model = Wav2Vec2ForCTC.from_pretrained("ai4bharat/indicwav2vec-hindi")
model.eval()
print("✅ Model loaded!\n")

audio_files = sorted(Path("test_audio").glob("*.wav"))
print(f"Found {len(audio_files)} files\n")
print("="*60)

for audio_path in audio_files:
    print(f"\n🎤 {audio_path.name}")
    
    # Load audio with torchaudio
    waveform, sample_rate = torchaudio.load(str(audio_path))
    
    # Resample to 16kHz if needed
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(sample_rate, 16000)
        waveform = resampler(waveform)
    
    # Convert to mono if stereo
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    
    # Squeeze to 1D
    waveform = waveform.squeeze().numpy()
    
    # Process
    input_values = processor(waveform, sampling_rate=16000, return_tensors="pt").input_values
    
    # Get prediction
    with torch.no_grad():
        logits = model(input_values).logits
    
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    
    # Show results
    txt_file = audio_path.with_suffix('.txt')
    if txt_file.exists():
        with open(txt_file, 'r', encoding='utf-8') as f:
            actual = f.read().strip()
        print(f"📝 Actual:    {actual}")
        print(f"🤖 Predicted: {transcription}")
    else:
        print(f"🤖 Predicted: {transcription}")
    
    print("-"*60)

print("\n✅ Done!")

Loading model...


/Users/krittinnagar/Speech Dubbing /.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


/Users/krittinnagar/Speech Dubbing /.venv/lib/python3.11/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(ch

✅ Model loaded!

Found 15 files


🎤 sample_1.wav
📝 Actual:    नमस्ते, मेरा नाम राज है
🤖 Predicted: नमस्ते मेरा नाम राज है
------------------------------------------------------------

🎤 sample_1_english_dubbed.wav
🤖 Predicted: माएं नई मिश्खिंडम
------------------------------------------------------------

🎤 sample_1_tamil_voice_cloned.wav
🤖 Predicted: न्याय संथी रेंठ्या डेना
------------------------------------------------------------

🎤 sample_2.wav
📝 Actual:    आज मौसम बहुत अच्छा है
🤖 Predicted: आज मौसम बहुत अच्छा है
------------------------------------------------------------

🎤 sample_2_english_dubbed.wav
🤖 Predicted: द वादरस वेडीकयुद्ध दे
------------------------------------------------------------

🎤 sample_2_tamil_voice_cloned.wav
🤖 Predicted: जरा कै जाय देखते हैं वृश कु गोद ए
------------------------------------------------------------

🎤 sample_3.wav
📝 Actual:    मुझे हिंदी बोलना पसंद है
🤖 Predicted: मुझे हिंदी बोलना पसंद है
----------------------------------------------------

In [4]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torchaudio
from pathlib import Path

# Load STT model
print("Loading Hindi STT model...")
stt_processor = Wav2Vec2Processor.from_pretrained("ai4bharat/indicwav2vec-hindi")
stt_model = Wav2Vec2ForCTC.from_pretrained("ai4bharat/indicwav2vec-hindi")
stt_model.eval()
print("✅ STT loaded!")

# Load Translation model (Hindi -> English)
print("\nLoading Hindi to English translation model...")
translation_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-hi-en")
translation_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-hi-en")
translation_model.eval()
print("✅ Translation model loaded!\n")

def translate_hindi_to_english(hindi_text):
    """Translate Hindi to English"""
    inputs = translation_tokenizer(hindi_text, return_tensors="pt", padding=True)
    with torch.no_grad():
        translated = translation_model.generate(**inputs, max_length=512)
    return translation_tokenizer.decode(translated[0], skip_special_tokens=True)

# Process audio files
audio_files = sorted(Path("test_audio").glob("*.wav"))
print("="*60)
print("AUDIO -> HINDI -> ENGLISH PIPELINE")
print("="*60 + "\n")

for audio_path in audio_files:
    print(f"🎤 {audio_path.name}")
    
    # Step 1: Audio -> Hindi
    waveform, sample_rate = torchaudio.load(str(audio_path))
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(sample_rate, 16000)
        waveform = resampler(waveform)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    waveform = waveform.squeeze().numpy()
    
    input_values = stt_processor(waveform, sampling_rate=16000, return_tensors="pt").input_values
    with torch.no_grad():
        logits = stt_model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    hindi_text = stt_processor.decode(predicted_ids[0])
    
    # Step 2: Hindi -> English
    english_text = translate_hindi_to_english(hindi_text)
    
    # Show results
    txt_file = audio_path.with_suffix('.txt')
    if txt_file.exists():
        with open(txt_file, 'r', encoding='utf-8') as f:
            original = f.read().strip()
        print(f"   Original: {original}")
    
    print(f"   Hindi:    {hindi_text}")
    print(f"   English:  {english_text}")
    print("-"*60)

print("\n✅ Pipeline complete!")
print("\nNext: We'll translate English -> Telugu")

Loading Hindi STT model...
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


Some weights of the model checkpoint at ai4bharat/indicwav2vec-hindi were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at ai4bharat/indicwav2vec-hindi and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this mode

✅ STT loaded!

Loading Hindi to English translation model...


/Users/krittinnagar/Speech Dubbing /.venv/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


✅ Translation model loaded!

AUDIO -> HINDI -> ENGLISH PIPELINE

🎤 sample_1.wav
   Original: नमस्ते, मेरा नाम राज है
   Hindi:    नमस्ते मेरा नाम राज है
   English:  Hello my name is the Kingdom.
------------------------------------------------------------
🎤 sample_1_english_dubbed.wav
   Hindi:    माएं नई मिश्खिंडम
   English:  Jess New Mundineum
------------------------------------------------------------
🎤 sample_1_tamil_voice_cloned.wav
   Hindi:    न्याय संथी रेंठ्या डेना
   English:  Judgment Day Afflictive Despera
------------------------------------------------------------
🎤 sample_2.wav
   Original: आज मौसम बहुत अच्छा है
   Hindi:    आज मौसम बहुत अच्छा है
   English:  The Weather Is Very Good Today
------------------------------------------------------------
🎤 sample_2_english_dubbed.wav
   Hindi:    द वादरस वेडीकयुद्ध दे
   English:  Give The Vedus Wedik War.
------------------------------------------------------------
🎤 sample_2_tamil_voice_cloned.wav
   Hindi:    जरा कै जाय

In [5]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torchaudio
from pathlib import Path

print("Loading Hindi STT model...")
stt_processor = Wav2Vec2Processor.from_pretrained("ai4bharat/indicwav2vec-hindi")
stt_model = Wav2Vec2ForCTC.from_pretrained("ai4bharat/indicwav2vec-hindi")
stt_model.eval()

translation_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
translation_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

def translate_hindi_to_english(hindi_text):
    """Translate Hindi to English"""
    translation_tokenizer.src_lang = "hi"  # Hindi source
    encoded = translation_tokenizer(hindi_text, return_tensors="pt")
    generated_tokens = translation_model.generate(
        **encoded,
        forced_bos_token_id=translation_tokenizer.get_lang_id("en")  # English
    )
    return translation_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

def translate_english_to_tamil(english_text):
    """Translate English to Tamil"""
    translation_tokenizer.src_lang = "en"  # English source
    encoded = translation_tokenizer(english_text, return_tensors="pt")
    generated_tokens = translation_model.generate(
        **encoded,
        forced_bos_token_id=translation_tokenizer.get_lang_id("ta")  # Tamil target
    )
    return translation_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

# Process audio files
audio_files = sorted(Path("test_audio").glob("*.wav"))


results = []

for audio_path in audio_files:
    
    waveform, sample_rate = torchaudio.load(str(audio_path))
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(sample_rate, 16000)
        waveform = resampler(waveform)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    waveform = waveform.squeeze().numpy()
    
    input_values = stt_processor(waveform, sampling_rate=16000, return_tensors="pt").input_values
    with torch.no_grad():
        logits = stt_model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    hindi_text = stt_processor.decode(predicted_ids[0])
    
    english_text = translate_hindi_to_english(hindi_text)
    
    tamil_text = translate_english_to_tamil(english_text)
    
    txt_file = audio_path.with_suffix('.txt')
    if txt_file.exists():
        with open(txt_file, 'r', encoding='utf-8') as f:
            original = f.read().strip()
    
    # Store results
    results.append({
        'file': audio_path.name,
        'original': original if txt_file.exists() else '',
        'hindi': hindi_text,
        'english': english_text,
        'tamil': tamil_text
    })

# Save results
import pandas as pd
df = pd.DataFrame(results)
df.to_csv("hindi_to_tamil_results.csv", index=False, encoding='utf-8-sig')
print(f"\n💾 Results saved to: hindi_to_tamil_results.csv")

print("\n" + "="*60)
print("PIPELINE SUMMARY")
print("="*60)
print("✅ Speech-to-Text: Hindi Audio -> Hindi Text")
print("✅ Translation 1:   Hindi Text -> English Text")
print("✅ Translation 2:   English Text -> Tamil Text")
print("\nNext step: Tamil Text -> Tamil Speech (TTS)")

Loading Hindi STT model...
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


Some weights of the model checkpoint at ai4bharat/indicwav2vec-hindi were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at ai4bharat/indicwav2vec-hindi and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this mode


💾 Results saved to: hindi_to_tamil_results.csv

PIPELINE SUMMARY
✅ Speech-to-Text: Hindi Audio -> Hindi Text
✅ Translation 1:   Hindi Text -> English Text
✅ Translation 2:   English Text -> Tamil Text

Next step: Tamil Text -> Tamil Speech (TTS)


In [6]:
import torch
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, AutoModel,
    Wav2Vec2ForCTC, Wav2Vec2Processor,
    M2M100ForConditionalGeneration, M2M100Tokenizer
)
import torchaudio
from pathlib import Path
import numpy as np
stt_processor = Wav2Vec2Processor.from_pretrained("ai4bharat/indicwav2vec-hindi")
stt_model = Wav2Vec2ForCTC.from_pretrained("ai4bharat/indicwav2vec-hindi")
stt_model.eval()

# IndicBERT for context
indicbert_tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBERTv2-MLM-only")
indicbert_model = AutoModel.from_pretrained("ai4bharat/IndicBERTv2-MLM-only")
indicbert_model.eval()

# Sentiment Analysis
sentiment_tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")
sentiment_model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")
sentiment_model.eval()

# Translation
translation_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
translation_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")


# Context analysis function
def analyze_context(text):
    """Analyze sentiment and context"""
    inputs = sentiment_tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
        scores = outputs.logits[0].softmax(dim=0).numpy()
    
    labels = ['Negative', 'Neutral', 'Positive']
    sentiment = labels[np.argmax(scores)]
    confidence = float(np.max(scores))
    
    # Get IndicBERT embeddings
    bert_inputs = indicbert_tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        bert_outputs = indicbert_model(**bert_inputs)
    
    return {
        'sentiment': sentiment,
        'confidence': confidence,
        'scores': {label: float(score) for label, score in zip(labels, scores)}
    }

# Translation functions
def translate(text, src_lang, tgt_lang):
    translation_tokenizer.src_lang = src_lang
    encoded = translation_tokenizer(text, return_tensors="pt")
    generated = translation_model.generate(
        **encoded,
        forced_bos_token_id=translation_tokenizer.get_lang_id(tgt_lang)
    )
    return translation_tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

# Process audio files

results = []

for audio_path in audio_files:
    print(f"🎤 {audio_path.name}")
    
    # Step 1: Audio -> Hindi
    waveform, sr = torchaudio.load(str(audio_path))
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000)
        waveform = resampler(waveform)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    
    input_values = stt_processor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt").input_values
    with torch.no_grad():
        logits = stt_model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    hindi_text = stt_processor.decode(predicted_ids[0])
    
    # Step 2: Context Analysis
    context = analyze_context(hindi_text)
    
    # Step 3: Translate
    english_text = translate(hindi_text, "hi", "en")
    tamil_text = translate(english_text, "en", "ta")
    
    # Show results
    txt_file = audio_path.with_suffix('.txt')
    if txt_file.exists():
        with open(txt_file, 'r', encoding='utf-8') as f:
            print(f"   Original:  {f.read().strip()}")
    
    print(f"   Hindi:     {hindi_text}")
    print(f"   📊 Context: {context['sentiment']} ({context['confidence']:.1%})")
    print(f"      Positive: {context['scores']['Positive']:.1%}, "
          f"Neutral: {context['scores']['Neutral']:.1%}, "
          f"Negative: {context['scores']['Negative']:.1%}")
    print(f"   English:   {english_text}")
    print(f"   Tamil:     {tamil_text}")
    print("-"*60)
    
    results.append({
        'file': audio_path.name,
        'hindi': hindi_text,
        'sentiment': context['sentiment'],
        'confidence': context['confidence'],
        'english': english_text,
        'tamil': tamil_text
    })

# Save results
import pandas as pd
df = pd.DataFrame(results)
df.to_csv("context_analysis_results.csv", index=False, encoding='utf-8-sig')


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


/Users/krittinnagar/Speech Dubbing /.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/krittinnagar/Speech Dubbing /.venv/lib/python3.11/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless th

🎤 sample_1.wav
   Original:  नमस्ते, मेरा नाम राज है
   Hindi:     नमस्ते मेरा नाम राज है
   📊 Context: Positive (47.7%)
      Positive: 47.7%, Neutral: 41.7%, Negative: 10.7%
   English:   My name is Kingdom.
   Tamil:     என் பெயர் அரசாங்கம்.
------------------------------------------------------------
🎤 sample_1_english_dubbed.wav
   Hindi:     माएं नई मिश्खिंडम
   📊 Context: Positive (43.2%)
      Positive: 43.2%, Neutral: 37.4%, Negative: 19.3%
   English:   The New Mishikandam
   Tamil:     புதிய Mishikandam
------------------------------------------------------------
🎤 sample_1_tamil_voice_cloned.wav
   Hindi:     न्याय संथी रेंठ्या डेना
   📊 Context: Neutral (56.0%)
      Positive: 15.6%, Neutral: 56.0%, Negative: 28.4%
   English:   Judgment by Danny.
   Tamil:     வழக்கறிஞர் டான்
------------------------------------------------------------
🎤 sample_2.wav
   Original:  आज मौसम बहुत अच्छा है
   Hindi:     आज मौसम बहुत अच्छा है
   📊 Context: Positive (81.5%)
      Positive: 81.5

In [10]:
# 🔹 Import all required libraries
import os
import pandas as pd
import torch
import soundfile as sf

from transformers import VitsModel, AutoTokenizer


# 🔹 Load model + tokenizer
model = VitsModel.from_pretrained("facebook/mms-tts-tam")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-tam")
model.eval()  # Set model to evaluation mode


# 🔹 Load CSV
df = pd.read_csv("hindi_to_tamil_results.csv")


tts_results = []

for index, row in df.iterrows():
    tamil_text = row["tamil"]
    file_name = row["file"].replace(".wav", "_tamil_tts.wav")
    output_path = os.path.join("test_audio", "tamil_audio", file_name)

    # Create output folder if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # 🔹 Generate speech
    inputs = tokenizer(tamil_text, return_tensors="pt")

    with torch.no_grad():
        audio_waveform = model(**inputs).waveform

    # 🔹 Save audio
    sf.write(
        output_path,
        audio_waveform.squeeze().numpy(),
        samplerate=model.config.sampling_rate
    )

    tts_results.append({
        'file': row['file'],
        'original_hindi': row['original'],
        'translated_english': row['english'],
        'translated_tamil': row['tamil'],
        'tamil_audio_path': output_path
    })


# 🔹 Save results with audio paths
tts_df = pd.DataFrame(tts_results)
tts_df.to_csv("tamil_tts_results.csv", index=False, encoding='utf-8-sig')


/Users/krittinnagar/Speech Dubbing /.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/mms-tts-tam were not used when initializing VitsModel: ['flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'posterior_encoder.wavenet.in_layers.7.weight_v', 'flow.flows.3.wavenet.res_skip_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.3.wavenet.in_layers.0.weight_v', 'posterior_encoder.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'posterior_encoder.wavenet.res_skip_layers.5.weight_g', 'posterior_encoder.wavenet.in_layers.1.weight_g', 'flow.flows.1.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.2.wavenet.res_skip

In [20]:
# Hindi video -> Tamil dubbed video pipeline (using Svara TTS v1)
import json
import subprocess
from pathlib import Path
import numpy as np
import torch
import torchaudio
import soundfile as sf
import requests
from transformers import (
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    M2M100ForConditionalGeneration,
    M2M100Tokenizer,
)

# Configuration
video_path = Path("assets/hindi_demo_clip_long.mp4")
output_dir = Path("demo_outputs")
output_dir.mkdir(parents=True, exist_ok=True)
clip_tag = video_path.stem

raw_audio_path = output_dir / f"{clip_tag}_audio.wav"
tamil_audio_path = output_dir / f"{clip_tag}_tamil.wav"
dubbed_video_path = output_dir / f"{clip_tag}_tamil_dub.mp4"
metadata_path = output_dir / f"{clip_tag}_metadata.json"

manual_offset_sec = 11.0

# Svara TTS API endpoint (local deployment)
SVARA_API_URL = "http://localhost:8080/v1/text-to-speech"

if not video_path.exists():
    raise FileNotFoundError(f"Input video not found at {video_path}")

print("🎬 Extracting audio track from Hindi video via ffmpeg...")
subprocess.run([
    "ffmpeg", "-y", "-i", str(video_path),
    "-ac", "1", "-ar", "16000", "-vn", str(raw_audio_path)
], check=True)

target_sr = 16000
waveform, sample_rate = torchaudio.load(str(raw_audio_path))
if sample_rate != target_sr:
    waveform = torchaudio.transforms.Resample(sample_rate, target_sr)(waveform)
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)
waveform_np = waveform.squeeze().numpy()
target_length = waveform_np.shape[-1]

# Load Hindi ASR model
if "stt_processor" not in globals():
    print("📥 Loading Hindi ASR model...")
    stt_processor = Wav2Vec2Processor.from_pretrained("ai4bharat/indicwav2vec-hindi")
if "stt_model" not in globals():
    stt_model = Wav2Vec2ForCTC.from_pretrained("ai4bharat/indicwav2vec-hindi")
    stt_model.eval()

print("🗣️ Running Hindi ASR...")
with torch.no_grad():
    input_values = stt_processor(waveform_np, sampling_rate=target_sr, return_tensors="pt").input_values
    logits = stt_model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
hindi_text = stt_processor.decode(predicted_ids[0])
print(f"   Detected Hindi text: {hindi_text}")

# Load translation model
if "translation_model" not in globals():
    print("📥 Loading translation model...")
    translation_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
    translation_model.eval()
if "translation_tokenizer" not in globals():
    translation_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

def translate_text(text: str, src_lang: str, tgt_lang: str) -> str:
    translation_tokenizer.src_lang = src_lang
    encoded = translation_tokenizer(text, return_tensors="pt")
    generated_tokens = translation_model.generate(
        **encoded, forced_bos_token_id=translation_tokenizer.get_lang_id(tgt_lang)
    )
    return translation_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

print("🌐 Translating Hindi -> English -> Tamil...")
english_text = translate_text(hindi_text, "hi", "en")
tamil_text = translate_text(english_text, "en", "ta")

# Manual override from metadata JSON
if metadata_path.exists():
    manual_meta = json.loads(metadata_path.read_text(encoding="utf-8"))
    english_text = manual_meta.get("english_translation", english_text)
    tamil_text = manual_meta.get("tamil_translation", tamil_text)

print(f"   English: {english_text}")
print(f"   Tamil  : {tamil_text}")

print("🔊 Generating Tamil speech with Svara TTS v1...")

# Using Svara TTS API
try:
    print("   Attempting to connect to Svara TTS API...")
    response = requests.post(
        SVARA_API_URL,
        json={
            "text": tamil_text,
            "voice_id": "ta_female",  # Tamil female voice
            "stream": True
        },
        stream=True,
        timeout=60
    )
    
    if response.status_code == 200:
        # Save PCM audio
        pcm_path = output_dir / f"{clip_tag}_tamil_temp.pcm"
        with open(pcm_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        # Convert PCM to WAV (Svara outputs at 24kHz)
        subprocess.run([
            "ffmpeg", "-y", "-f", "s16le", "-ar", "24000", "-ac", "1",
            "-i", str(pcm_path), "-ar", str(target_sr), str(tamil_audio_path)
        ], check=True)
        
        pcm_path.unlink()
        
        # Load the generated audio
        tamil_waveform, tamil_sr = torchaudio.load(str(tamil_audio_path))
        if tamil_sr != target_sr:
            tamil_waveform = torchaudio.transforms.Resample(tamil_sr, target_sr)(tamil_waveform)
        tamil_waveform_np = tamil_waveform.squeeze().numpy()
        
        print("   ✓ Successfully generated audio via Svara TTS API")
        tts_method = "Svara TTS v1 API"
        
    else:
        raise Exception(f"API returned status code {response.status_code}")
        
except Exception as e:
    print(f"   ⚠️ API connection failed: {e}")
    print("   Falling back to gTTS...")
    
    from gtts import gTTS
    gtts_mp3 = output_dir / f"{clip_tag}_gtts.mp3"
    gtts_wav = output_dir / f"{clip_tag}_gtts.wav"
    
    gtts = gTTS(text=tamil_text, lang="ta")
    gtts.save(gtts_mp3)
    
    subprocess.run([
        "ffmpeg", "-y", "-i", str(gtts_mp3),
        "-ac", "1", "-ar", str(target_sr), str(gtts_wav)
    ], check=True)
    
    tamil_waveform, tamil_sr = torchaudio.load(str(gtts_wav))
    if tamil_sr != target_sr:
        tamil_waveform = torchaudio.transforms.Resample(tamil_sr, target_sr)(tamil_waveform)
    tamil_waveform_np = tamil_waveform.squeeze().numpy()
    
    sf.write(str(tamil_audio_path), tamil_waveform_np, samplerate=target_sr)
    tts_method = "gTTS (fallback)"
    
    gtts_mp3.unlink(missing_ok=True)
    gtts_wav.unlink(missing_ok=True)

# Align audio with video
aligned_waveform = np.zeros(target_length, dtype=np.float32)
offset_samples = min(int(manual_offset_sec * target_sr), target_length - 1)
trimmed_length = min(target_length - offset_samples, len(tamil_waveform_np))
aligned_waveform[offset_samples:offset_samples + trimmed_length] = tamil_waveform_np[:trimmed_length]

sf.write(str(tamil_audio_path), aligned_waveform, samplerate=target_sr)

print("🎞️ Replacing video audio track with Tamil dub via ffmpeg...")
subprocess.run([
    "ffmpeg", "-y", "-i", str(video_path), "-i", str(tamil_audio_path),
    "-c:v", "copy", "-map", "0:v:0", "-map", "1:a:0", str(dubbed_video_path)
], check=True)

metadata = {
    "source_video": str(video_path),
    "hindi_transcript": hindi_text,
    "english_translation": english_text,
    "tamil_translation": tamil_text,
    "tamil_audio": str(tamil_audio_path),
    "dubbed_video": str(dubbed_video_path),
    "start_offset_sec": manual_offset_sec,
    "tts_engine": tts_method,
    "model_name": "kenpath/svara-tts-v1",
    "voice_id": "ta_female",
}
metadata_path.write_text(json.dumps(metadata, ensure_ascii=False, indent=2), encoding="utf-8")

print("✅ Tamil dubbed video ready! Saved to", dubbed_video_path)
print("✅ Metadata saved to", metadata_path)
print(f"✅ TTS method used: {tts_method}")

🎬 Extracting audio track from Hindi video via ffmpeg...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.0.13.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab

🗣️ Running Hindi ASR...
   Detected Hindi text: आनिया मुखयाचज्या विका ी गुनी शिक्षा वन के बप उपर तेशोाकण सबभने नाहस का आज मै प्रदूषिण के कारण औ नरवारण इस विषय पर अपने विचार क्करना चाहती हों पुराने समाने में ह ा में पचा सा ग होते तीनचासौ लोग गायवे बकरे हुते मुर्गी मयना कहोतर चर और घरे बरे के त पौधे मदीमाले इस तरब जी बिंतु अपनस का क्े में संतुलन था खेतों में फसल अच्छा होता का जीवन सुख मे था लेकिन आप कन न्या बहुत
🌐 Translating Hindi -> English -> Tamil...
   English: Three hundred and a hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one hundred and one

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.0.13.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab

🎞️ Replacing video audio track with Tamil dub via ffmpeg...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.0.13.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab

✅ Tamil dubbed video ready! Saved to demo_outputs/hindi_demo_clip_long_tamil_dub.mp4
✅ Metadata saved to demo_outputs/hindi_demo_clip_long_metadata.json
✅ TTS method used: gTTS (fallback)


[out#0/mp4 @ 0x1246060d0] video:1226KiB audio:377KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 1.468320%
frame=  720 fps=0.0 q=-1.0 Lsize=    1627KiB time=00:00:59.83 bitrate= 222.7kbits/s speed= 259x elapsed=0:00:00.23    
[aac @ 0x12460a720] Qavg: 63460.449
